# Confusion Matrix - SVM (AE Min 8 Node)

In [1]:
import numpy as np
import pickle
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def one_hot_convert_normal(data, end_line, output_num):
    list_data = []

    for row in range(0, end_line):
        for column in range(0, output_num):
            if data[row][column] == 1:
                list_data.append(column + 1)
    return list_data

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

In [6]:
# Training data
output_num = 10
end_line = train_set_num * output_num

oh_y_in_train_denoised = one_hot_convert_normal(y_in_train_denoised, end_line, output_num)
oh_y_in_train_denoised = np.array(oh_y_in_train_denoised)

oh_y_out_train_denoised = one_hot_convert_normal(y_out_train_denoised, end_line, output_num)
oh_y_out_train_denoised = np.array(oh_y_out_train_denoised)

# Testing data
end_line = test_set_num * output_num

oh_y_in_test_denoised = one_hot_convert_normal(y_in_test_denoised, end_line, output_num)
oh_y_in_test_denoised = np.array(oh_y_in_test_denoised)

oh_y_out_test_denoised = one_hot_convert_normal(y_out_test_denoised, end_line, output_num)
oh_y_out_test_denoised = np.array(oh_y_out_test_denoised)

In [7]:
autoencoder_01 = load_model('autoencoder_01.h5')
autoencoder_02 = load_model('autoencoder_02.h5')
autoencoder_03 = load_model('autoencoder_03.h5')
autoencoder_04 = load_model('autoencoder_04.h5')

# [ 방법 1 ]
## 데이터 10개의 평균 사용

In [8]:
import numpy as np

def average(x, y, set_num, data_num):
    idx = 0
    sum = np.array([0, 0, 0])
    data = [[0 for col in range(3)] for row in range(int(set_num * 10 / data_num))]
    label = [[0 for col in range(10)] for row in range(int(set_num * 10 / data_num))]
    for i in range(set_num * 10):
        for j in range(3):
            sum[j] += x[i][j]
        if(i % data_num == data_num - 1):
            for j in range(3):
                data[idx][j] = int(round(sum[j] / data_num))
            for j in range(10):
                label[idx][j] = y[i][j]
            idx += 1
            sum = np.array([0, 0, 0])    
    return data, label

In [9]:
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy) * (-100)
x_train, y_train = average(after_autoencoder_x_train, y_in_train_noisy, 4500, 10)

In [10]:
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy) * (-100)
x_test, y_test = average(after_autoencoder_x_test, y_in_test_noisy, 1500, 10)

In [11]:
x_train = np.array(x_train)
x_test = np.array(x_test)
ae_x_in_train = x_train.astype('float32') / -100.
ae_x_in_test = x_test.astype('float32') / -100.

In [12]:
# Average data
# Train data
oh_y_in_train = one_hot_convert_normal(y_train, int(4500 * output_num / 10), output_num)
oh_y_in_train = np.array(oh_y_in_train)

# Test data
oh_y_in_test = one_hot_convert_normal(y_test, int(1500 * output_num / 10), output_num)
oh_y_in_test = np.array(oh_y_in_test)

## SVM 01

In [13]:
svc_01 = pickle.load(open('svc_01.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = svc_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = svc_01.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = svc_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = svc_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = svc_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = svc_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]
 [4459    0    0    0    0    0   38    0    0    3]
 [4500    0    0    0    0    0    0    0    0    0]
 [4500    0    0    0    0    0    0    0    0    0]]


Testing data Confusion Matrix
[[1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1500    0    0    0    0    0    0    0    0    0]
 [1488    0    0    0    0

## SVM 02

In [14]:
svc_02 = pickle.load(open('svc_02.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = svc_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = svc_02.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = svc_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = svc_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = svc_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = svc_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[4384   75    0   11    0    0   30    0    0    0]
 [  28 4273    0    0    0    0   84    0    0  115]
 [   0    0 4500    0    0    0    0    0    0    0]
 [ 140    0    0 4262   85    0    0    0   13    0]
 [   0    0    0   44 4261   19    0    0  176    0]
 [   0    0    0    0    3 4373    0    0  124    0]
 [   7   69    0    0    0    0 4424    0    0    0]
 [   0    0    0    0    0    0    0 4500    0    0]
 [   0    0    6   69  220  161    0    4 4040    0]
 [   0  202    0    0    0    0    0    0    0 4298]]


Testing data Confusion Matrix
[[1460   11    0    9    0    0    7    0   13    0]
 [  12 1406    0    0    0    0   30    0   10   42]
 [   0    0 1500    0    0    0    0    0    0    0]
 [  49    0    0 1402   39    0    0    0   10    0]
 [   0    0    0   16 1416    3    0    0   65    0]
 [   0    0    0    0    2 1448    0    0   50    0]
 [   1   24    0    0    0    0 1472    0    3    0]
 [   0    0    0    0    0

## SVM 03

In [15]:
svc_03 = pickle.load(open('svc_03.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = svc_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = svc_03.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = svc_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = svc_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = svc_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = svc_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[2437   44    0    0    0    0    0    0    0 2019]
 [  41 3522    0    0    0    0    0    0    0  937]
 [   0  129 3273    0    5    0    0    0    0 1093]
 [   0  113    0  972   62    0    0    0    7 3346]
 [   0   89    0  193 1811    9    0    0    1 2397]
 [   0  126    0    0   13 1495    0    0    0 2866]
 [   0   11    0    0    0    0  818    0    0 3671]
 [   0   57    0    0    0    0    0  802    0 3641]
 [   0    3    0   17   26   14    0    0 1136 3304]
 [  23  351    0    0    0    0    0    0    0 4126]]


Testing data Confusion Matrix
[[ 785   11    0    0    0    0    0    0    0  704]
 [  23 1173    0    0    0    0    0    0    0  304]
 [   0   42 1074    0    1    0    0    0    0  383]
 [   0   30    0  349   42    0    0    0    4 1075]
 [   0   39    0   51  613    3    0    0    0  794]
 [   0   33    0    0    4  466    0    0    0  997]
 [   0    3    0    0    0    0  245    0    0 1252]
 [   0   11    0    0    0

## SVM 04

In [16]:
svc_04 = pickle.load(open('svc_04.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = svc_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = svc_04.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = svc_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = svc_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = svc_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = svc_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = svc_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[  83   44 4373    0    0    0    0    0    0    0]
 [   4 3999  497    0    0    0    0    0    0    0]
 [   0    0 4500    0    0    0    0    0    0    0]
 [   0    4 4490    0    6    0    0    0    0    0]
 [   0    0 4313    0  187    0    0    0    0    0]
 [   0    0 4500    0    0    0    0    0    0    0]
 [   0   18 4482    0    0    0    0    0    0    0]
 [   0    0 4500    0    0    0    0    0    0    0]
 [   0    0 4486    0    6    0    0    0    8    0]
 [   0  340 4160    0    0    0    0    0    0    0]]


Testing data Confusion Matrix
[[  18   13 1469    0    0    0    0    0    0    0]
 [   0 1327  173    0    0    0    0    0    0    0]
 [   0    0 1500    0    0    0    0    0    0    0]
 [   0    0 1498    0    2    0    0    0    0    0]
 [   0    0 1435    0   65    0    0    0    0    0]
 [   0    0 1500    0    0    0    0    0    0    0]
 [   0    5 1495    0    0    0    0    0    0    0]
 [   0    0 1500    0    0